In [ ]:
import numpy as np
import xarray as xr
import pickle
import pandas as pd
import time
import os

import datetime
from sys import getsizeof,path


import matplotlib.pyplot as plt
import matplotlib

import theano
import theano.tensor as tt
import pymc3 as pm
theano.config.optimizer="fast_run"

In [ ]:
path.append("../src")
import Bernstein
import Population
import Cases

# Import age_stratified Casees from RKI source
5y Age groups up to 90+ are available, unfortunately no sex / Bundesländer.

In [ ]:
age_cases10y,age_cases5y = Cases.RKI_Altersverteilung()
age_cases5y.coords

In [ ]:
# Sum of all cases per age Group
max_week = max(age_cases5y.get_index("week"))
latest_cases = age_cases5y.sel(publication=age_cases5y.get_index("publication")[0]).sel(week=range(10,max_week+1))

plt.plot(latest_cases.get_index("age"),latest_cases.sum("week") ) 
plt.title("Cases in 5y agegroups")

print(max_week)

In [ ]:
cases_adj = latest_cases.sel(age=[70,75,85,90])
cases_80 = latest_cases.sel(age=80)

rel = cases_adj/cases_80

plt.plot(rel.get_index("week"),rel.transpose("week","age"))
plt.legend(rel.get_index("age").values)
plt.title("Age Groups relative to 80-84")
plt.xlabel("week of 2020")

plt.savefig("plots/AgeGroupsRelativeTo80_byWeeks.svg")

# Next Step: 1y age resolution
Let's take a look at actual 1y age resolution 0-79 and the group 80+

In [ ]:
with open("../Data/Cases/SurvStat_RKI/lks_210210.pickle","br") as f:
    cases = Cases.ParseSurvStatDay_BL(pickle.load(f))

In [ ]:
cases1y = cases.sum(["category","BL","sex"])
cases1y.coords

In [ ]:
cases1y_ws80 = cases1y.sum("week").sel(age=80)
cases1y_ws = cases1y.sel(week=[20,36,42,44,52,53]).sel(age=range(61,80))
plt.plot(cases1y_ws.get_index("age"),cases1y_ws/cases1y_ws.sel(age=79))
plt.legend(cases1y_ws.get_index("week"))

plt.title("Age distribution relative to 79")
plt.xlabel("age")

In [ ]:
w10 = np.array([1/(x*x) for x in range(1,11)[::-1]],dtype="float64")[:,np.newaxis]
f10 = np.sum( (cases1y_ws.sel(age=range(70,80))*w10) ,axis=0)

plt.plot(cases1y_ws.get_index("age"),cases1y_ws/f10)
plt.legend(cases1y_ws.get_index("week"))

plt.title("Age distribution relative to 70-79, weighted 1/x^2")
plt.xlabel("age")

In [ ]:
# Now scale 80+